#CA02 Code Explanation: Elisabeth Webb

##Naive Bayes Classification Algorithm for Spam Email Detection

The goal is to determine if a set of emails is spam or not spam. There are 702 emails divided equally into spam and not spam. This is considered supervised learning, as we already know which emails are spam and not spam. We will test the model on 260 emails to determine if it can correctly classify them. The model will predict the category each email should be in, and then we will compare the accuracy score with the correct classification. 

# Cleaning and Preparing the Data

In [0]:
#import libraries
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [0]:
#In order to clean and prepare the text data, we need to remove expressions, symbols, and stop words, 
#which are those that are not meaningful and don't contribute to the analysis.
#To determine which words to keep, we will use the 3000 most common words from the emails.  

def create_Dictionary(root_mail_dir): #creates a dictionary that goes through the all the email and adds together the total words and symbols.
  total_words = [] #creates an empty list of all the words in the each of the emails put together
  all_emails = [os.path.join(root_mail_dir,v) for v in os.listdir(root_mail_dir)] #gets the emails from the path and puts it onto a list
  for email in all_emails: #for each email in the list above of all emails
    with open(email) as message: #open the email 
      for line in message: #for each line in the message
        words = line.split() #splits the lines in each message by word and stores in new variable named words
        total_words += words #does the split for every message and adds all the words together 
  dictionary = Counter(total_words) #keeps track of how many times the same word was added 
  words_to_remove = list(dictionary) #creates a list of words that are duplicated and need to be removed later
 
#The for loop removes the characters (non-numeric alpha and single character alpha-numeric) in the dictionary created above
#We want to keep only the alphabetical words 
  for item in words_to_remove: #for each item in the list of words that need to be removed
    if item.isalpha() == False: #if the item in the string is not alphabetical then (non-numeric alpha)
      del dictionary[item] #delete that item
    elif len(item) == 1: #if the item if a single character alpha-numeric word
      del dictionary[item] #then delete that item as well
  dictionary = dictionary.most_common(3000) #new dictionary that keeps the 3000 most common words after non-required characters have been removed
  return dictionary #display the dictionary to see what words are in it
#create_Dictionary(TRAIN_MAIL) #run the function on the sets of emails used for training the model
#create_Dictionary(TEST_MAIL) #run the function on the sets of emails used for testing the model


# Extracting features and corresponding label matrix

In [0]:
#The function created below extracts and removes words that aren't as common
#It makes the common words as columns and the total number of emails the rows in the matrix
#It creates the matrix by extracting words for the columns and counting how many time those words appear in each email
#The result is a matrix showing the 3000 most common words and frequency of those words in each email

#The function looks at each email's file name and determines if it's spam or not spam based on how it is named 
#The function also creates a labelled data column
#The function is used to on the training and testing data and will return the feature dataset and the labelled data column

def extract_elements(email_dir): #dictionary will extract the elements from the directory of emails
  email_files = [os.path.join(email_dir,fi) for fi in os.listdir(email_dir)] #gets the files from the path and adds them to list
  features_matrix = np.zeros((len(email_files),3000)) #creates an array with the shape of 3000 words for columns and the number of how many email files for rows
  train_labels = np.zeros(len(email_files)) #creates an array shape specifially for the emails used for training the model
  count = 1; #the count of words will be in the columns axis=1 is columns 
  emailID = 0; #the email identified by it's ID will in shown in the rows of the matrix axis=0 means rows
  for file in email_files: #for file in email_files list 
    with open(file) as fi: #open the file as fi 
      for i, line in enumerate(fi): #for each line in every file, give every item (named i) it's own column
        if i ==2: #if the item (named i) is two words in one
          words = line.split() #then split the line and store in a words variable
          for word in words: #for every word in the the words variable
            wordID = 0 #create new variable that will be able to uniquely identify the word and start at 0 and will be added to
            for i, v in enumerate(dictionary): #goes every item (i) and value (v) in the dictionary of 3000 common words
              if v[0] == word: #if the first value is a word 
                wordID = i #identify evey item in the dictionary as its own word and give it a wordID
                features_matrix[emailID,wordID] = words.count(word) #count every word as many times as it appear. In the feature matrix, the value of each word in every email (identified by their ID) will equal to that count.
      train_labels[emailID] = 0; #when training the model, start every emailID for their word count at 0 and then it will add to it when counting 
      filepathTokens = file.split('/') #splits every file by '/' 
      lastToken = filepathTokens[len(filepathTokens)-1] #find the length of every file in filepathTokens and the last one use as the lastToken
      if lastToken.startswith("spmsg"): #tests to see if the lastToken ends with "spmsg"
        train_labels[emailID] = 1; #labels every unique email in the training email set as 1, meaning its own row
        count = count + 1 #every time a word appears in that email it adds one to it 
      emailID = emailID + 1 #every time an emailID appears in the list it add one to to it to keep all emails unqiue and only represented once 
  return features_matrix, train_labels #returns the result
#extract_features(TRAIN_MAIL) #run the function on the sets of emails used for training the model
#extract_features(TEST_MAIL) #run the function on the sets of emails used for testing the model

# Training and predicting with sklearn Naive Bayes

In [16]:
#This is where the program executes the two functions created above
#First, model.fit trains the model using the training dataset
#Next, it runs the trained model with the test dataset and uses that to score the test dataset 
#Lastly, it prints the model accuracy score to show how well the model preformed

TRAIN_MAIL = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails' #path for the train dataset
TEST_MAIL = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails' #path for the test dataset

dictionary = create_Dictionary(TRAIN_MAIL) #applies the first function created about on the training mail dataset

print ("reading and processing emails from TRAIN and TEST folders") #reads that it is processing
features_matrix, labels = extract_elements(TRAIN_MAIL) #applies the second function on training mail dataset
test_features_matrix, test_labels = extract_elements(TEST_MAIL) #applies the second function on the testing mail dataset

model = GaussianNB() #uses the Guassian Naive Bayes function on the model that will fit the data and create the feature matrix and labelled data column

print ("Training Model using Gaussian Naibe Bayes algorithm .....") #prints that it is running the Gaussian function
model.fit(features_matrix, labels) #the Gaussian model fits the data to the feature matrix and labels, which is the cleanred trained email dataset
print ("Training completed") #says when it's done
print ("testing trained model to predict Test Data labels") #says that it will now used the trained model to test the data labels and predict which categories they will be in
predicted_labels = model.predict(test_features_matrix) #the Gaussian model is fitting to the cleaned test mail and predicting the classification of emails

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels


# Accuracy Score

In [17]:
#After predicting whether an email will be spam or not, we compare the model's result with the actual labels
#The result is 96% of emails from the model were acturately predicted

#Explains that the model classification has been completed on the set of emails used for testing the model
#then says what is being printed, which is the accuracy score
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels)) #prints the accuracy of the model compared to the actual labels
#the accuracy score of the model can be used to evaluate its effectiveness at predicting and categorizing spam and non-spam emails

Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616


# Conclusion

###The Naive Bayes algorithm was able to classify emails into spam and not spam based on the words they contain. Naive Bayes considers independent features. For example, it assumes that each word is independent compared to the other words. This is a key part of how the algorithm's results work, but assuming indpendence might not always be the case. For instance, the words good and bye or good and morning might have a strong chance of occuring together in greetings or emails, so you should take this into considering when interpreting the results of Naive Bayes.
